In [ ]:
# Copyright 2019 Institut für Nachrichtentechnik, RWTH Aachen University
%matplotlib notebook

from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from IPython.display import clear_output, display, HTML

import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import scipy as sp
import scipy.special # erfc(x)

from ient_nb.ient_plots import *
from ient_nb.ient_signals import rect

eps = np.finfo(float).eps # very small positive number

<div>
    <img src="figures/rwth_ient_logo@2x.png" style="float: right;height: 5em;">
</div>

# Quantisierung

Exemplarisches Signal $s(t) = 2\sin(2\pi t)$

In [ ]:
x = np.linspace(-3, 3, 5001) # x-axis
t = np.arange(-.2, 1.1, 0.0001) # t-axis

Amax = 2
s = Amax*np.sin(2*np.pi*t)

## Gleichförmige Quantisierung

### Midtreat-Quantisierung:

Kennlinie
$$k(x) = \left\lfloor x +\frac{1}{2} \right\rfloor$$

In [ ]:
# Characteristic
k = lambda x: np.floor(x+0.5)

# Quantize
sQ = k(s)

# Plot
fig,axs = plt.subplots(1,2, figsize=(8,4)); 
ax = axs[0]; ax.plot(x, k(x), 'rwth'); 
ient_annotate_distance(ax, r'$\Delta$', [1,0], [1,1]); ax.texts[1].set_position((5,5));
ax.set_xlabel(r'$\rightarrow x$'); ax.set_ylabel(r'$\uparrow k(x)$');
ax.set_xlim([-2.25, 2.25]); ax.set_ylim([-2.25, 2.25]); ient_grid(ax); ient_axis(ax); 

ax = axs[1]; ax.plot(t, s, 'k--', label=r'$s(t)$'); ax.plot(t, sQ, 'rwth', label=r'$s_\mathrm{Q}(t)$')
ax.set_xlabel(r'$\rightarrow t$'); 
ax.set_ylim(axs[0].get_ylim()); ax.legend(); ient_grid(ax); ient_axis(ax);

### Midrise-Quantisierung:

Kennlinie
$$k(x) = \frac{1}{2} + \left\lfloor x \right\rfloor$$

In [ ]:
# Characteristic
k = lambda x: 0.5+np.floor(x)

# Quantize
sQ = k(s)

# Plot
fig,axs = plt.subplots(1,2, figsize=(8,4)); 
ax = axs[0]; ax.plot(x, k(x), 'rwth'); 
ax.set_xlabel(r'$\rightarrow x$'); ax.set_ylabel(r'$\uparrow k(x)$');
ax.set_xlim([-1.75, 2.75]); ax.set_ylim([-1.75, 2.75]); ient_grid(ax); ient_axis(ax);

ax = axs[1]; ax.plot(t, s, 'k--', label=r'$s(t)$'); ax.plot(t, sQ, 'rwth', label=r'$s_\mathrm{Q}(t)$')
ax.set_xlabel(r'$\rightarrow t$'); 
ax.set_ylim(axs[0].get_ylim()); ax.legend(); ient_grid(ax); ient_axis(ax);

### Generalisierung

Stufenhöhe jetzt variabel (vorher $\Delta=1$). Mit frei wählbarer Anzahl an Quantisierungsstufen $N$ ergibt sich für Stufenhöhe
$$\Delta = \frac{2 A_\max}{N-1}\text{.}$$
und rein positive Signale $\Delta = \frac{A_\max}{N-1}$.

Kennlinien
* Midtreat 
    $$k(x) = \Delta\left\lfloor \frac{x}{\Delta} +\frac{1}{2} \right\rfloor$$
* Midrise
    $$k(x) = \Delta\left[\frac{1}{2} + \left\lfloor \frac{x}{\Delta} \right\rfloor\right]$$

In [ ]:
# Characteristics
k_cl = lambda x, Amax:  np.clip(x, -Amax, Amax) # clip
k_mt = lambda x, delta: delta*(np.floor( k_cl(x, Amax)/delta+0.5 ))  # midtreat
k_mr = lambda x, delta: delta*(np.floor( k_cl(x, Amax)/delta ) + 0.5)# midrise

Quantisierungsfehler
$$s_\mathrm{D}(t) = s_\mathrm{Q}(t)-s(t)$$

mit Quantisierungsfehlerkennlinie
$q(x) = k(x)-x$.

Ohne Übersteuerung und mit kleiner Stufenhöhe $\Delta$ gilt
$$p_{s_\mathrm{D}}(z) = \frac{1}{\Delta}\mathrm{rect}\left(\frac{z}{\Delta}\right) $$
wobei Quantisierungsstörleistung
$$N_\mathrm{Q} = \mathcal{E}\{s_\mathrm{D}^2(t)\} =\frac{\Delta^2}{12}$$

Interaktive Demo

In [ ]:
fig,axs = plt.subplots(2,2, figsize=(8,8)); 
@interact(N = widgets.IntSlider(value=7, min=2,max=20),
          qtype = widgets.RadioButtons(options=['Midtreat', 'Midrise'], 
                                       description='Quantization type:', style=ient_wdgtl_style))
def update_plot(N, qtype):
    # Quantize
    delta = 2*Amax/(N-1)
    if qtype == 'Midtreat':
        sQ = k_mt(s, delta); xQ = k_mt(x, delta);
    else:
        sQ = k_mr(s, delta); xQ = k_mr(x, delta);
    # Quantization error
    sD = sQ-s; xD = xQ-x; # quantization error
    psD = rect(x/delta)/delta # pdf of error
    psD_hist, bins = np.histogram(sD, bins='auto', range=(-Amax, Amax), density=True)
    x_hist = (bins[:-1] + bins[1:])/2 # x-axis
    
    if not axs[0,0].lines:
        ax = axs[0,0]; ax.plot(x, xQ, 'rwth'); 
        ax.set_xlabel(r'$\rightarrow x$'); ax.set_ylabel(r'$\uparrow k(x)$');
        ax.axis('equal'); ient_grid(ax); ax.set_xlim([-2.4, 2.4]); ient_axis(ax); 
        ax = axs[0,1]; ax.plot(t, s, 'k--', label=r'$s(t)$'); ax.plot(t, sQ, 'rwth', label=r'$s_\mathrm{Q}(t)$')
        ax.set_xlabel(r'$\rightarrow t$'); ax.legend(); ient_grid(ax); ient_axis(ax);
        ax = axs[1,0]; ax.plot(x, xD, 'rwth');
        ax.set_xlabel(r'$\rightarrow x$'); ax.set_ylabel(r'$\uparrow q(x) = k(x)-x$', bbox=ient_wbbox); 
        ient_grid(ax); ient_axis(ax);
        ax = axs[1,1]; ient_stem(ax, x_hist, psD_hist, 'black-50', markerfmt=" "); ax.plot(x, psD, 'rwth')
        ax.set_xlabel(r'$\rightarrow z$'); ax.set_ylabel(r'$\uparrow p_{s_\mathrm{D}}(z)$'); 
        ax.set_ylim([0, 4.9]); ient_grid(ax); ient_axis(ax);
    else:
        axs[0,0].lines[0].set_ydata(xQ); axs[0,1].lines[1].set_ydata(sQ)
        axs[1,0].lines[0].set_ydata(xD); axs[1,1].lines[-1].set_ydata(psD)
        ient_stem_set_data(axs[1,1].containers[0], x_hist, psD_hist)
        
    print("Nq (gemessen): {:.3f} ({:.3f})".format(delta**2/12, np.mean((s-sQ)**2)))

This notebook is provided as [Open Educational Resource](https://en.wikipedia.org/wiki/Open_educational_resources) (OER). Feel free to use the notebook for your own purposes. The code is licensed under the [MIT license](https://opensource.org/licenses/MIT). 

Please attribute the work as follows: 
*Christian Rohlfing, Übungsbeispiele zur Vorlesung "Informationsübertragung"*, gehalten von Jens-Rainer Ohm, 2019, Institut für Nachrichtentechnik, RWTH Aachen University.